## Setup and imports

In [1]:
%pip install pyarrow

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import json
from pathlib import Path
import pickle
import time

from IPython.display import display, clear_output
import numpy as np
import pandas as pd
import requests

%matplotlib inline

# for i in range(10):
#     clear_output(wait=True)
#     print(i)
#     time.sleep(0.1)

DATA_DIR = Path('./data')
UNRETRACTED_DIR = DATA_DIR / 'json_collecting'
tester = DATA_DIR / 'test.txt'
tester.touch()
assert tester.exists()

PER_PAGE = 200 # API max
SHARED_FILTERS = "type:journal-article,publication_year:>2010,publication_year:<2018"

## Now to collate

In [3]:
with Path(DATA_DIR / 'raw_retracted.json').open('r', encoding='UTF-8') as infile:
    raw_records = json.load(infile)

# keep only (retracted) records with valid ['host_venue']['id']
# -- since we're matching year/journal we could have done this first but it hadn't occurred to me
print(len(raw_records))
records = [r for r in raw_records if r['host_venue']['id']]
print(len(records))
del raw_records
# 9,549 -> 9,546

9549
9546


In [5]:
# turns out this is a real memory issue
MAX_UNRETRACTED_RECORDS_PER = 20

# make sure we don't run this twice; but I don't want to make a copy and burn memory
assert len(records) == 9546 

# then append the new unretracted jsons -- remember these were straight saved and include the 'meta' 
unretracted_file_list = list(UNRETRACTED_DIR.glob('*.json'))
n_unretracted = len(unretracted_file_list)
for i, path in enumerate(unretracted_file_list):
    clear_output(wait=True)
    print(i+1, 'of', n_unretracted, path)
    with path.open('r', encoding='UTF-8') as infile:
        j = json.load(infile)
        result_subset = j['results'][:MAX_UNRETRACTED_RECORDS_PER]
        records += result_subset
    print(f'...now {len(records)} records.')

raw_collated_path = DATA_DIR / 'raw_collated.json'    
# json is only about 20% inflated over pkl
with raw_collated_path.open('w', encoding='UTF-8') as outfile:
    json.dump(records, outfile)

5259 of 5259 data/json_collecting/V175922936-2013.json
...now 113937 records.


In [6]:
# fresh start with no records, memory is at 0.2 GB
# with n =   9,546 in records, memory is at 0.7 GB (MAX_UNRETRACTED_RECORDS_PER = 0)
# with n =  20,000 in records, memory is at 1.1 GB (MAX_UNRETRACTED_RECORDS_PER = 2)
# with n =  61,995 in records, memory is at 3.1 GB (MAX_UNRETRACTED_RECORDS_PER = 10)
# with n = 113,937 in records, memory is at 5.3 GB (MAX_UNRETRACTED_RECORDS_PER = 20)

# 113,937 records is ~1.2 GB on disk.

In [14]:
import os
print(round(os.path.getsize(raw_collated_path) / (1024*1024*1024), 1), 'gb')

1.1 gb
